In [1]:
import fitsio
import numpy as np
from tvtk.api import tvtk, write_data

In [43]:
vxf = '/home/rattie/Data/flow_samples/vx_fwhm15_tavg160_000.fits' 
vyf = '/home/rattie/Data/flow_samples/vy_fwhm15_tavg160_000.fits'  
u = 368000/60
vx = fitsio.read(vxf)*u
vy = fitsio.read(vyf)*u
dim = vx.shape


vx = np.moveaxis(np.array([vx.T, np.zeros(dim)]), 0, 2)
vy = np.moveaxis(np.array([vy.T, np.zeros(dim)]), 0, 2)
vz = np.zeros(vx.shape)

dim=vx.shape
vx.shape, vy.shape, vz.shape

((512, 512, 2), (512, 512, 2), (512, 512, 2))

In [44]:
xx,yy,zz=np.mgrid[0:dim[0],0:dim[1],0:dim[2]]
pts = np.empty(vx.shape + (3,), dtype=int)
pts[..., 0] = xx
pts[..., 1] = yy
pts[..., 2] = zz

vectors = np.empty(vx.shape + (3,), dtype=float)
vectors[..., 0] = vx
vectors[..., 1] = vy
vectors[..., 2] = vz


In [45]:
# We reorder the points and vectors so this is as per VTK's
# requirement of x first, y next and z last.
pts = pts.transpose(2, 1, 0, 3).copy()
pts.shape = pts.size // 3, 3

In [46]:
vectors = vectors.transpose(2, 1, 0, 3).copy()
vectors.shape = vectors.size // 3, 3

In [47]:
sg = tvtk.StructuredGrid(dimensions=xx.shape, points=pts)

sg.point_data.vectors = vectors
sg.point_data.vectors.name = 'velocity'

write_data(sg, 'vtktest_t_u.vtk')